# Figure 3 - Layout

In [ ]:
import sys
sys.path.append("..")

from main import *
from visualization import *

from scipy.signal import find_peaks
from scipy.stats import spearmanr

# Loading data for figure

In [ ]:
h_values = np.linspace(0.025, 0.6, 100, endpoint=True)
correlations_h = np.load('../ComputeCanada/kernel_size/mode_correlations_per_h_2500.npy')

avg_correlations_h = np.mean(np.abs(correlations_h), axis=1)

diagonals = []
for a in avg_correlations_h:
    diagonals.append(np.diag(a))
diagonals = np.stack(diagonals, axis=1)

variograms = np.load('../Results/mode_variograms.npy')
wavelengths = np.load('../Results/mode_wavelengths.npy')

cutoffs = np.load('../Results/figure3_cutoffs_kernelsize.npy')

correlations_swaps = np.load('../ComputeCanada/edge_swapping/mode_correlations_per_rho_swaps.npy')
fractions = np.linspace(0, 0.99, 36, endpoint=True)
avg_correlations_swaps = np.mean(np.abs(correlations_swaps), axis=1)

diagonals_swaps = []
for a in avg_correlations_swaps:
    diagonals_swaps.append(np.diag(a))
diagonals_swaps = np.stack(diagonals_swaps, axis=1)

cutoffs_swaps = np.load('../Results/figure3_cutoffs_swaps.npy')

profiles_kernelsize = np.load('../Results/figure3_profiles_kernelsize.npy')
profiles_aligned_kernelsize = np.load('../Results/figure3_aligned_profiles_kernelsize.npy')
profiles_edgeswaps = np.load('../Results/figure3_profiles_edgeswaps.npy')
profiles_aligned_edgeswaps = np.load('../Results/figure3_aligned_profiles_edgeswaps.npy')

# Rendering figure

In [ ]:
red = plt.get_cmap('Reds')(175)
blue = plt.get_cmap('Blues')(175)

In [ ]:
fig = PaperFigure(figsize=(3.5, 7))

fig.set_tick_length(2)
fig.set_font_size(6)
fig.add_background()

# Setting up axes ----------------------------------------------------------------------------------


w = 0.65
pad = (3.5 - 5 * w) / 4
linepad = 0.5

fig.add_axes('corrs1', (0 * (w + pad), 0), w, w)
fig.add_axes('corrs2', (1 * (w + pad), 0), w, w)
fig.add_axes('corrs3', (2 * (w + pad), 0), w, w)
fig.add_axes('corrs4', (3 * (w + pad), 0), w, w)
fig.add_axes('corrs5', (4 * (w + pad), 0), w, w)
fig.add_axes('mode_decay_h', (0, w + linepad), 2 * w + pad, 1.5 * w)
fig.add_axes('scatter_h', (3.5 - (1.5 * w), w + linepad), 1.5 * w, 1.5 * w)

fig.add_axes('corrs1_swaps', (0 * (w + pad), 2.5 * w + 2 * linepad), w, w)
fig.add_axes('corrs2_swaps', (1 * (w + pad), 2.5 * w + 2 * linepad), w, w)
fig.add_axes('corrs3_swaps', (2 * (w + pad), 2.5 * w + 2 * linepad), w, w)
fig.add_axes('corrs4_swaps', (3 * (w + pad), 2.5 * w + 2 * linepad), w, w)
fig.add_axes('corrs5_swaps', (4 * (w + pad), 2.5 * w + 2 * linepad), w, w)

fig.add_axes('mode_decay_swaps', (0,  3.5 * w + 3 * linepad), 2 * w + pad, 1.5 * w)
fig.add_axes('scatter_swaps', (3.5 - (1.5 * w),  3.5 * w + 3 * linepad), 1.5 * w, 1.5 * w)

y = 5 * w + 4 * linepad
w = 0.95
pad = (3.5 - 3 * w) / 2
fig.add_axes('profiles1', (0 * (w + pad), y), w, w)
fig.add_axes('profiles2', (1 * (w + pad), y), w, w)
fig.add_axes('profiles3', (2 * (w + pad), y), w, w)

fig.set_line_thickness(0.6)

# Filling content --------------------------------------------------------------------------

matrix_ids = [12, 16, 20, 30, 50]
for i in range(5):
    ax = fig.axes['corrs{}'.format(i + 1)]
    ax.imshow(avg_correlations_h[matrix_ids[i]], cmap='Reds', vmin=0.1, vmax=1)
    ax.set_xticks([])
    ax.set_yticks([])

ax = fig.axes['mode_decay_h']
ax.imshow(diagonals, cmap='Reds', aspect='auto', vmin=0, vmax=1)
ax.set_xticks([0, 50, 99], [0, 0.3, 0.6])

ax = fig.axes['scatter_h']
x = np.array(cutoffs[1:])
y = np.array(wavelengths[2:51])
ax.scatter(x, y, s=1, color='black')
ax.spines[['top', 'right']].set_visible(False)
a, b = np.polyfit(x, y, deg=1)
ax.plot(x, a * x + b, linewidth=1, color='red')
ax.set_ylim([0.2, 1.1])

matrix_ids_swaps = [0, 4, 8, 12, 16]
for i in range(5):
    ax = fig.axes['corrs{}_swaps'.format(i + 1)]
    ax.imshow(avg_correlations_swaps[matrix_ids_swaps[i]], cmap='Blues', vmin=0.1, vmax=1)
    ax.set_xticks([])
    ax.set_yticks([])

ax = fig.axes['mode_decay_swaps']
ax.imshow(diagonals_swaps, cmap='Blues', aspect='auto', vmin=0, vmax=1)
ax.set_xticks([0, 17.5, 35], [0, 0.5, 1.0])

ax = fig.axes['scatter_swaps']
x = cutoffs_swaps[1:]
y = np.array(wavelengths[2:51]) # Excluding first trivial eigenmode, and second eigenmode whose wavelength is not properly measured
ax.scatter(x, y, s=1, color='black')
ax.spines[['top', 'right']].set_visible(False)
a_, b = np.polyfit(x, y, deg=1)
ax.plot(x, a_ * x + b, linewidth=1, color=blue)
#a, b = np.polyfit(cutoffs_swaps, np.array(wavelengths[1:51]), deg=1)
#ax.plot(cutoffs_swaps, a * np.array(cutoffs_swaps) + b, linewidth=1, color='red')

ax = fig.axes['profiles1']
ax.spines[['top', 'right']].set_visible(False)
for d in profiles_kernelsize:
    ax.plot(h_values, d, color=red, alpha=0.2, linewidth=0.5)
ax.set_xlim([np.min(h_values), 0.6])

ax = fig.axes['profiles2']
ax.spines[['top', 'right']].set_visible(False)
for d in profiles_edgeswaps:
    ax.plot(fractions, d, color=blue, alpha=0.2, linewidth=0.5)
ax.set_xlim([0, 1])
ax.set_yticks([])

ax = fig.axes['profiles3']
ax.spines[['top', 'right']].set_visible(False)
for c in profiles_aligned_edgeswaps:
    ax.plot(np.linspace(0, 1, len(c)), c, color=blue, linewidth=0.5, alpha=0.2)
#ax.plot(np.linspace(0, 1, len(c)), np.nanmean(profiles_aligned_edgeswaps, axis=0), color=blue, linewidth=1, alpha=1)
for c in profiles_aligned_kernelsize:
    ax.plot(np.linspace(0, 1, len(c)), c, color=red, linewidth=0.5, alpha=0.2)
#ax.plot(np.linspace(0, 1, len(c)), np.nanmean(profiles_aligned_kernelsize, axis=0), color=red, linewidth=1, alpha=1)
ax.set_yticks([])
ax.set_xlim([0, 1])

fig.show()

In [ ]:
fig.save('../Figures/figure3_incomplete.svg')

# Manually annotated values

Printing numbers to manually annotate the figure with.

In [ ]:
a

In [ ]:
h_values[matrix_ids]

In [ ]:
pearsonr(cutoffs[1:], 0.5 * np.array(wavelengths[2:51]))[0] ** 2

In [ ]:
pearsonr(cutoffs_swaps[1:], 0.5 * np.array(wavelengths[2:51]))[0] ** 2

In [ ]:
h_values[15] - h_values[0]

In [ ]:
fractions[15] - fractions[0]

# Supplementary figure S6: Line profiles

In [ ]:
fig = PaperFigure(figsize=(7, 1.5))

fig.set_tick_length(2)
fig.set_font_size(6)
fig.add_background()

# Setting up axes ----------------------------------------------------------------------------------

w = 1.75
pad = (7 - 3 * w) / 2
fig.add_axes('profiles1', (0 * (w + pad), 0), w, 1.25)
fig.add_axes('profiles2', (1 * (w + pad), 0), w, 1.25)
fig.add_axes('profiles3', (2 * (w + pad), 0), w, 1.25)

fig.set_line_thickness(0.6)

# Filling content --------------------------------------------------------------------------

ax = fig.axes['profiles1']
ax.spines[['top', 'right']].set_visible(False)
for d in profiles_kernelsize:
    ax.plot(h_values, d, color=red, alpha=0.2, linewidth=0.5)
ax.set_xlim([np.min(h_values), 0.6])

ax = fig.axes['profiles2']
ax.spines[['top', 'right']].set_visible(False)
for d in profiles_edgeswaps:
    ax.plot(fractions, d, color=blue, alpha=0.2, linewidth=0.5)
ax.set_xlim([0, 1])
#ax.set_yticks([])

ax = fig.axes['profiles3']
ax.spines[['top', 'right']].set_visible(False)
for c in profiles_aligned_edgeswaps:
    ax.plot(np.linspace(0, 1, len(c)), c, color=blue, linewidth=0.5, alpha=0.2)
#ax.plot(np.linspace(0, 1, len(c)), np.nanmean(profiles_aligned_edgeswaps, axis=0), color=blue, linewidth=1, alpha=1)
for c in profiles_aligned_kernelsize:
    ax.plot(np.linspace(0, 1, len(c)), c, color=red, linewidth=0.5, alpha=0.2)
#ax.plot(np.linspace(0, 1, len(c)), np.nanmean(profiles_aligned_kernelsize, axis=0), color=red, linewidth=1, alpha=1)
#ax.set_yticks([])
ax.set_xlim([0, 1])

fig.show()

In [ ]:
fig.save('../Figures/supp_lineprofiles_incomplete.svg')